# Field Mapping dengan ADVANCED Cross-Table Enrichment
## Post-processing enrichment: ambil nilai dari transaksi untuk isi master data kosong

In [49]:
# Cell 1: Setup
import requests, hashlib, hmac, base64, json, time, os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pytz
import warnings
warnings.filterwarnings('ignore')

API_TOKEN = "aat.NTA.eyJ2IjoxLCJ1IjoxMDIyNDE2LCJkIjo5NDY3OTMsImFpIjo2MDMxMiwiYWsiOiIwOGRlZmNiMC1kNjEzLTQxYjgtOGI5YS0zOWNhNjQ1OWIzOTkiLCJhbiI6IkFwbGlrYXNpIC0gRGF0YSBDb2xsZWN0aW9uIiwiYXAiOiI2NzgwZTA1YS0wNjQ3LTQ2NzktYmEyYi1jMWE4YWEyZGZjYWUiLCJ0IjoxNzYwMDkwNzI4OTcwfQ.LemzKJp8Tgp+yacEUvUM8hgTrUbb2rhCgNrpW/WsznGtvusfjeVV7AkqPShw0QvqL4bUey3k7BbifqwJVtTVAFp84BfyrC0/YwM7Xl5zycmf95dsJZV8we1yD13KRDcG5PoBCqh5Y4CY0oz39gBPM5oMcy9PZixjYKSc8/LaqfMMZLfaYMPuGjb5ppq9KbLVqFWQSbheqRc=.McqkDx7gdPa9Fzn501K/Fsfzzb8N7iF08un74VZqQaA"
SIGNATURE_SECRET = "VdQuYB9APtdyJxgFOGr8CtSMUtjVjmeTxDRhnrnOuh9el8qft2h5RO61ftO1Zr5l"
wib = pytz.timezone('Asia/Jakarta')
end_date = datetime.now().strftime('%d/%m/%Y')
start_date = (datetime.now() - timedelta(days=90)).strftime('%d/%m/%Y')

def find_column(df, *possible_names):
    if df.empty: return None
    cols_lower = {c.lower(): c for c in df.columns}
    for name in possible_names:
        if name.lower() in cols_lower:
            return cols_lower[name.lower()]
    return None

def safe_float(val, default=0.0):
    try: return float(val) if val is not None and str(val).lower() not in ['nan','none',''] else default
    except: return default

def safe_str(val, default=''):
    return str(val) if val is not None and str(val).lower() not in ['nan','none',''] else default

def enrich_value(primary, *fallbacks):
    if primary is not None and str(primary).lower() not in ['nan','none','','0','0.0']:
        return primary
    for fb in fallbacks:
        if fb is not None and str(fb).lower() not in ['nan','none','','0','0.0']:
            return fb
    return primary if primary is not None else (fallbacks[0] if fallbacks else '')

def enrich_numeric(primary, *fallbacks):
    val = safe_float(primary)
    if val > 0: return val
    for fb in fallbacks:
        fval = safe_float(fb)
        if fval > 0: return fval
    return val

print(f'📅 Period: {start_date} - {end_date}')

📅 Period: 23/09/2025 - 22/12/2025


In [50]:
# Cell 2: API Client
class AccurateAPI:
    def __init__(self, token, secret):
        self.token, self.secret, self.host = token, secret, None
        self.req_count, self.last_req = 0, time.time()
    def sign(self, ts): return base64.b64encode(hmac.new(self.secret.encode(), ts.encode(), hashlib.sha256).digest()).decode()
    def headers(self):
        ts = datetime.now(wib).strftime('%d/%m/%Y %H:%M:%S')
        return {'Authorization': f'Bearer {self.token}', 'X-Api-Timestamp': ts, 'X-Api-Signature': self.sign(ts), 'Content-Type': 'application/json'}
    def rate_limit(self):
        if time.time() - self.last_req >= 1: self.req_count, self.last_req = 0, time.time()
        if self.req_count >= 3: time.sleep(max(1.2 - (time.time() - self.last_req), 0)); self.req_count, self.last_req = 0, time.time()
        self.req_count += 1
    def init(self):
        r = requests.post('https://account.accurate.id/api/api-token.do', headers=self.headers())
        d = r.json(); k = 'database' if 'database' in d.get('d',{}) else 'data usaha'
        if d.get('s'): self.host = d['d'][k]['host']; print(f'✅ Connected: {self.host}'); return True
        return False
    def get(self, ep, p=None):
        if not self.host: return None
        for i in range(4):
            self.rate_limit()
            try:
                r = requests.get(f'{self.host}/accurate{ep}', headers=self.headers(), params=p, timeout=30)
                if r.status_code == 429: time.sleep(2**i + 5); continue
                return r.json()
            except: time.sleep(2**i)
        return None
    def pages(self, ep, p=None, mx=50):
        p = p or {}; data, pg = [], 1
        while pg <= mx:
            p['sp.page'], p['sp.pageSize'] = pg, 100
            r = self.get(ep, p)
            if not r or not r.get('s') or not r.get('d',[]): break
            data.extend(r['d'])
            if pg >= r.get('sp',{}).get('pageCount',1): break
            pg += 1; time.sleep(0.5)
        return data

client = AccurateAPI(API_TOKEN, SIGNATURE_SECRET)
client.init()

✅ Connected: https://iris.accurate.id


True

In [51]:
# Cell 3: Pull Master Data
print('='*70)
print('📦 PULLING MASTER DATA')
print('='*70)

print('\n📦 Items...')
df_items = pd.DataFrame(client.pages('/api/item/list.do', {}, 50))
print(f'   ✅ {len(df_items)} records, Columns: {list(df_items.columns)}')

print('\n🏭 Warehouses...')
df_wh = pd.DataFrame(client.pages('/api/warehouse/list.do', {}, 10))
print(f'   ✅ {len(df_wh)} records')

print('\n👥 Customers...')
df_cust = pd.DataFrame(client.pages('/api/customer/list.do', {}, 30))
print(f'   ✅ {len(df_cust)} records')

print('\n🏪 Vendors...')
df_vend = pd.DataFrame(client.pages('/api/vendor/list.do', {}, 30))
print(f'   ✅ {len(df_vend)} records')

📦 PULLING MASTER DATA

📦 Items...
   ✅ 2266 records, Columns: ['id']

🏭 Warehouses...
   ✅ 7 records

👥 Customers...
   ✅ 877 records

🏪 Vendors...
   ✅ 107 records


In [52]:
# Cell 4: Build Maps
print('🗺️ Building maps...')

id_col = find_column(df_items, 'id', 'itemId')
no_col = find_column(df_items, 'no', 'itemNo')
name_col = find_column(df_items, 'name', 'itemName')
type_col = find_column(df_items, 'itemType', 'type')
cat_col = find_column(df_items, 'itemCategoryName', 'categoryName')
cost_col = find_column(df_items, 'avgCost', 'cost')
price_col = find_column(df_items, 'unitPrice', 'sellingPrice')
unit_col = find_column(df_items, 'unit1Name', 'unitName')

prod_map, prod_by_no = {}, {}
for _, r in df_items.iterrows():
    pid = r.get(id_col) if id_col else None
    if pid:
        data = {col: r.get(col) for col in df_items.columns}
        data.update({'_id': pid, '_no': safe_str(r.get(no_col) if no_col else ''),
                     '_name': safe_str(r.get(name_col) if name_col else ''),
                     '_type': safe_str(r.get(type_col) if type_col else ''),
                     '_category': safe_str(r.get(cat_col) if cat_col else ''),
                     '_avgCost': safe_float(r.get(cost_col) if cost_col else 0),
                     '_unitPrice': safe_float(r.get(price_col) if price_col else 0),
                     '_unit1': safe_str(r.get(unit_col) if unit_col else '')})
        prod_map[pid] = data
        if data['_no']: prod_by_no[data['_no']] = data

wh_map, cust_map, vend_map = {}, {}, {}
for _, r in df_wh.iterrows():
    wid = r.get(find_column(df_wh, 'id'))
    if wid: wh_map[wid] = {col: r.get(col) for col in df_wh.columns}
for _, r in df_cust.iterrows():
    cid = r.get(find_column(df_cust, 'id'))
    if cid: cust_map[cid] = {col: r.get(col) for col in df_cust.columns}
for _, r in df_vend.iterrows():
    vid = r.get(find_column(df_vend, 'id'))
    if vid: vend_map[vid] = {col: r.get(col) for col in df_vend.columns}

print(f'✅ Maps: Products:{len(prod_map)}, WH:{len(wh_map)}, Cust:{len(cust_map)}, Vend:{len(vend_map)}')

🗺️ Building maps...
✅ Maps: Products:2265, WH:7, Cust:877, Vend:107


In [53]:
# Cell 5: Pull PO Details
print('='*70)
print('🛒 PURCHASE ORDER DETAILS')
print('='*70)

pos = client.pages('/api/purchase-order/list.do', {'filter.transDate.>=': start_date, 'filter.transDate.<=': end_date}, 30)
print(f'Processing {len(pos)} POs (limit 150)...')

po_details = []
for i, po in enumerate(pos[:150]):
    print(f'{i+1}/150...', end='\r')
    r = client.get('/api/purchase-order/detail.do', {'id': po['id']})
    if r and r.get('s'):
        d = r['d']
        k = next((x for x in ['detailItem','items','detail'] if x in d and d[x]), None)
        if k:
            for it in d[k]:
                item_id = it.get('itemId', it.get('id'))
                prod = prod_map.get(item_id, prod_by_no.get(it.get('itemNo'), {}))
                qty = enrich_numeric(it.get('quantity'))
                unit_price = enrich_numeric(it.get('unitPrice'), it.get('price'), prod.get('_unitPrice'))
                total_price = enrich_numeric(it.get('amount'), it.get('totalPrice'))
                if total_price == 0 and qty > 0 and unit_price > 0:
                    total_price = qty * unit_price
                po_details.append({
                    'item_no': enrich_value(it.get('itemNo'), prod.get('_no')),
                    'item_name': enrich_value(it.get('itemName'), prod.get('_name')),
                    'quantity': qty,
                    'unit_price': unit_price,
                    'total_price': total_price,
                    'po_number': po.get('number'),
                    'vendor_name': po.get('vendorName'),
                })
    time.sleep(0.3)

df_po = pd.DataFrame(po_details)
print(f'\n✅ PO Details: {len(df_po)} records')

🛒 PURCHASE ORDER DETAILS
Processing 1882 POs (limit 150)...
150/150...
✅ PO Details: 481 records


In [54]:
# Cell 6: Pull Sales Details  
print('='*70)
print('💰 SALES DETAILS')
print('='*70)

invs = client.pages('/api/sales-invoice/list.do', {'filter.transDate.>=': start_date, 'filter.transDate.<=': end_date}, 50)
print(f'Processing {len(invs)} invoices (limit 150)...')

sales_details = []
for i, inv in enumerate(invs[:150]):
    print(f'{i+1}/150...', end='\r')
    r = client.get('/api/sales-invoice/detail.do', {'id': inv['id']})
    if r and r.get('s'):
        d = r['d']
        k = next((x for x in ['detailItem','items','detail'] if x in d and d[x]), None)
        if k:
            for it in d[k]:
                item_id = it.get('itemId')
                prod = prod_map.get(item_id, prod_by_no.get(it.get('itemNo'), {}))
                qty = enrich_numeric(it.get('quantity'))
                unit_price = enrich_numeric(it.get('unitPrice'), prod.get('_unitPrice'))
                total_price = enrich_numeric(it.get('totalPrice'), it.get('amount'))
                if total_price == 0 and qty > 0 and unit_price > 0:
                    total_price = qty * unit_price
                sales_details.append({
                    'item_no': enrich_value(it.get('itemNo'), prod.get('_no')),
                    'item_name': enrich_value(it.get('itemName'), prod.get('_name')),
                    'quantity': qty,
                    'unit_price': unit_price,
                    'total_price': total_price,
                    'number': inv.get('number'),
                    'customer_name': inv.get('customerName'),
                })
    time.sleep(0.3)

df_sales = pd.DataFrame(sales_details)
print(f'\n✅ Sales: {len(df_sales)} records')

💰 SALES DETAILS
Processing 5000 invoices (limit 150)...
150/150...
✅ Sales: 318 records


In [55]:
# Cell 7: Pull Stock Mutations
print('='*70)
print('🔄 STOCK MUTATIONS')
print('='*70)

if type_col:
    inv_items = df_items[df_items[type_col].isin(['INVENTORY','GROUP'])]
else:
    inv_items = df_items
print(f'Processing {min(100, len(inv_items))} items...')

mutations = []
for i, (_, item) in enumerate(inv_items.head(100).iterrows()):
    print(f'{i+1}/100...', end='\r')
    item_id = item.get(id_col) if id_col else item.get('id')
    r = client.get('/api/item/stock-mutation-history.do', {'id': item_id, 'startDate': start_date, 'endDate': end_date})
    if r and r.get('s'):
        prod = prod_map.get(item_id, {})
        avg_cost = enrich_numeric(prod.get('_avgCost'))
        for rec in r.get('d', []):
            increase = enrich_numeric(rec.get('quantityIn'), rec.get('increase'))
            decrease = enrich_numeric(rec.get('quantityOut'), rec.get('decrease'))
            value_in = enrich_numeric(rec.get('valueIn'))
            if value_in == 0 and increase > 0: value_in = increase * avg_cost
            value_out = enrich_numeric(rec.get('valueOut'))
            if value_out == 0 and decrease > 0: value_out = decrease * avg_cost
            mutations.append({
                'item_no': enrich_value(item.get(no_col) if no_col else '', prod.get('_no')),
                'item_name': enrich_value(item.get(name_col) if name_col else '', prod.get('_name')),
                'trans_date': rec.get('transDate'),
                'increase': increase,
                'value_in': value_in,
                'decrease': decrease,
                'value_out': value_out,
            })
    time.sleep(0.3)

df_mutations = pd.DataFrame(mutations)
print(f'\n✅ Mutations: {len(df_mutations)} records')

🔄 STOCK MUTATIONS
Processing 100 items...
100/100...
✅ Mutations: 2000 records


In [56]:
# Cell 8: Pull Current Stock
print('='*70)
print('📊 CURRENT STOCK')
print('='*70)

stks = client.pages('/api/item/list-stock.do', {}, 50)
df_stock_raw = pd.DataFrame(stks)
print(f'Stock records: {len(df_stock_raw)}')

current_stock = []
for _, s in df_stock_raw.iterrows():
    item_id = s.get('id', s.get('itemId'))
    prod = prod_map.get(item_id, {})
    on_stock = enrich_numeric(s.get('qtyStock'), s.get('stockAvailable'))
    avg_cost = enrich_numeric(s.get('avgCost'), prod.get('_avgCost'))
    unit_price = enrich_numeric(s.get('unitPrice'), prod.get('_unitPrice'))
    
    current_stock.append({
        'item_no': enrich_value(s.get('no'), prod.get('_no')),
        'item_name': enrich_value(s.get('name'), prod.get('_name')),
        'on_stock': on_stock,
        'avg_cost': avg_cost,
        'unit_price': unit_price,
        'stock_value': on_stock * avg_cost,
    })

df_stock = pd.DataFrame(current_stock)
print(f'✅ Stock: {len(df_stock)} records')

📊 CURRENT STOCK
Stock records: 2060
✅ Stock: 2060 records


In [57]:
# Cell 9: ADVANCED POST-PROCESSING ENRICHMENT
print('='*70)
print('🔧 POST-PROCESSING CROSS-TABLE ENRICHMENT')
print('='*70)

# Build price lookup dari PO & Sales
print('\n📊 Building price lookup from transactions...')
po_prices = df_po[df_po['unit_price'] > 0].groupby('item_no')['unit_price'].agg(['mean', 'max', 'min', 'count']).reset_index()
sales_prices = df_sales[df_sales['unit_price'] > 0].groupby('item_no')['unit_price'].agg(['mean', 'max', 'min', 'count']).reset_index()

print(f'   PO prices: {len(po_prices)} items')
print(f'   Sales prices: {len(sales_prices)} items')

# Enrich Current Stock dengan harga dari transaksi
print('\n🔧 Enriching Current Stock...')
before_zero = (df_stock['unit_price'] == 0).sum()
print(f'   Before: {before_zero} items with unit_price=0')

for idx, row in df_stock.iterrows():
    if row['unit_price'] == 0 or pd.isna(row['unit_price']):
        item_no = row['item_no']
        
        # Cari dari sales terlebih dahulu (harga jual lebih akurat)
        sales_match = sales_prices[sales_prices['item_no'] == item_no]
        if not sales_match.empty:
            df_stock.at[idx, 'unit_price'] = sales_match['mean'].values[0]
            continue
        
        # Fallback ke PO price
        po_match = po_prices[po_prices['item_no'] == item_no]
        if not po_match.empty:
            df_stock.at[idx, 'unit_price'] = po_match['mean'].values[0]
            continue

# Recalculate stock_value setelah enrichment
df_stock['stock_value'] = df_stock['on_stock'] * df_stock.apply(
    lambda x: x['avg_cost'] if x['avg_cost'] > 0 else x['unit_price'], axis=1
)

after_zero = (df_stock['unit_price'] == 0).sum()
print(f'   After: {after_zero} items with unit_price=0')
print(f'   ✅ Enriched: {before_zero - after_zero} items')

# Enrich PO Details dengan avg_cost jika kosong
print('\n🔧 Enriching PO Details with avg_cost...')
for idx, row in df_po.iterrows():
    if (row.get('avg_cost', 0) == 0 or pd.isna(row.get('avg_cost'))):
        item_no = row['item_no']
        # Cari di stock
        stock_match = df_stock[df_stock['item_no'] == item_no]
        if not stock_match.empty and stock_match['avg_cost'].values[0] > 0:
            if 'avg_cost' not in df_po.columns:
                df_po['avg_cost'] = 0.0
            df_po.at[idx, 'avg_cost'] = stock_match['avg_cost'].values[0]

# Enrich Sales Details
print('\n🔧 Enriching Sales Details with cost data...')
for idx, row in df_sales.iterrows():
    item_no = row['item_no']
    stock_match = df_stock[df_stock['item_no'] == item_no]
    if not stock_match.empty:
        if 'avg_cost' not in df_sales.columns:
            df_sales['avg_cost'] = 0.0
        if 'margin' not in df_sales.columns:
            df_sales['margin'] = 0.0
        
        cost = stock_match['avg_cost'].values[0]
        df_sales.at[idx, 'avg_cost'] = cost
        # Calculate margin
        if cost > 0 and row['unit_price'] > 0:
            df_sales.at[idx, 'margin'] = ((row['unit_price'] - cost) / row['unit_price']) * 100

print('\n✅ Cross-table enrichment complete!')

🔧 POST-PROCESSING CROSS-TABLE ENRICHMENT

📊 Building price lookup from transactions...
   PO prices: 1 items
   Sales prices: 1 items

🔧 Enriching Current Stock...
   Before: 2060 items with unit_price=0
   After: 2060 items with unit_price=0
   ✅ Enriched: 0 items

🔧 Enriching PO Details with avg_cost...

🔧 Enriching Sales Details with cost data...

✅ Cross-table enrichment complete!


In [58]:
# Cell 10: Export dengan Quality Check Detail
print('='*70)
print('💾 EXPORTING WITH DETAILED QUALITY CHECK')
print('='*70)

csv_folder = 'field_mapping_csv_output'
os.makedirs(csv_folder, exist_ok=True)

exports = [
    ('1_Sales_Details', df_sales),
    ('2_PO_Details', df_po),
    ('3_Stock_Mutations', df_mutations),
    ('4_Current_Stock', df_stock),
    ('5_Master_Items', df_items),
]

with pd.ExcelWriter('field_mapping_output_project1.xlsx', engine='openpyxl') as w:
    for name, df in exports:
        if not df.empty:
            print(f'\n📊 {name} ({len(df)} rows, {len(df.columns)} cols)')
            print('-'*60)
            
            # Quality check ALL columns
            for col in df.columns:
                null_pct = df[col].isna().sum() / len(df) * 100
                if df[col].dtype in ['int64','float64']:
                    zero_pct = (df[col] == 0).sum() / len(df) * 100
                    non_zero = len(df) - (df[col] == 0).sum()
                    status = '✅' if (null_pct < 10 and zero_pct < 40) else ('⚠️' if null_pct < 30 else '❌')
                    print(f'   {status} {col:<25} null:{null_pct:5.1f}% zero:{zero_pct:5.1f}% (non-zero:{non_zero})')
                else:
                    empty = (df[col].astype(str).str.strip() == '').sum() + df[col].isna().sum()
                    empty_pct = empty / len(df) * 100
                    status = '✅' if empty_pct < 10 else ('⚠️' if empty_pct < 30 else '❌')
                    print(f'   {status} {col:<25} empty:{empty_pct:5.1f}%')
            
            df.to_excel(w, sheet_name=name[:31], index=False)
            df.to_csv(f'{csv_folder}/{name}.csv', index=False, encoding='utf-8-sig')

print('\n🎉 Export Complete!')
print(f'   📁 Excel: field_mapping_output_project1.xlsx')
print(f'   📁 CSV: {csv_folder}/')

💾 EXPORTING WITH DETAILED QUALITY CHECK

📊 1_Sales_Details (318 rows, 7 cols)
------------------------------------------------------------
   ❌ item_no                   empty:100.0%
   ❌ item_name                 empty:100.0%
   ✅ quantity                  null:  0.0% zero:  0.0% (non-zero:318)
   ✅ unit_price                null:  0.0% zero: 15.7% (non-zero:268)
   ✅ total_price               null:  0.0% zero: 15.7% (non-zero:268)
   ❌ number                    empty:100.0%
   ❌ customer_name             empty:100.0%

📊 2_PO_Details (481 rows, 7 cols)
------------------------------------------------------------
   ❌ item_no                   empty:100.0%
   ❌ item_name                 empty:100.0%
   ✅ quantity                  null:  0.0% zero:  0.0% (non-zero:481)
   ✅ unit_price                null:  0.0% zero:  1.2% (non-zero:475)
   ✅ total_price               null:  0.0% zero:  1.2% (non-zero:475)
   ❌ po_number                 empty:100.0%
   ❌ vendor_name               empty: